In [27]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.optimizers import SGD,Adam,RMSprop
from keras.utils import to_categorical
from keras import regularizers
from keras.utils import np_utils


In [28]:
train = pd.read_csv('train_real.csv')
test = pd.read_csv('test_real.csv')
sample_sub = pd.read_csv('sample_submission(1).csv')

In [29]:
submission_df = pd.DataFrame()
submission_df['Id'] = test['Id']

In [30]:
train.head()

,Id,Altitude,Azimuthal_angle,Incline,H_dist_Hydro,V_dist_Hydro,H_dist_Road,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,H_dist_Fire,Wilderness_Area_A,Wilderness_Area_B,Wilderness_Area_C,Wilderness_Area_D,Soil,label
0,1,594,738,167,48,243,421,759,964,751,114,0,0,1,0,Soil_Type_23,-1
1,2,343,103,212,43,239,25,861,818,480,208,0,0,1,0,Soil_Type_10,-1
2,3,563,233,167,77,228,555,928,869,452,360,1,0,0,0,Soil_Type_1,-1
3,4,715,602,136,193,289,175,830,979,684,164,0,0,1,0,Soil_Type_32,-1
4,5,672,519,136,231,292,122,869,971,625,45,0,0,1,0,Soil_Type_32,-1


In [31]:
sample_sub.head()

,Id,Predictions
0,1,-1
1,2,-1
2,3,-1
3,4,-1
4,5,-1


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12395 entries, 0 to 12394
Data columns (total 17 columns):
Id                   12395 non-null int64
Altitude             12395 non-null int64
Azimuthal_angle      12395 non-null int64
Incline              12395 non-null int64
H_dist_Hydro         12395 non-null int64
V_dist_Hydro         12395 non-null int64
H_dist_Road          12395 non-null int64
Hillshade_9am        12395 non-null int64
Hillshade_Noon       12395 non-null int64
Hillshade_3pm        12395 non-null int64
H_dist_Fire          12395 non-null int64
Wilderness_Area_A    12395 non-null int64
Wilderness_Area_B    12395 non-null int64
Wilderness_Area_C    12395 non-null int64
Wilderness_Area_D    12395 non-null int64
Soil                 12395 non-null object
label                12395 non-null int64
dtypes: int64(16), object(1)
memory usage: 1.6+ MB


In [33]:
#Importing LabelEncoder from Sklearn

from sklearn.preprocessing import LabelEncoder
label_encoder_soil = LabelEncoder()

In [34]:
# Transforming sex column values using label Encoder

train.iloc[:,15]  = label_encoder_soil.fit_transform(train.iloc[:,15])
test.iloc[:,15]  = label_encoder_soil.fit_transform(test.iloc[:,15])

In [35]:
train = train.drop(['Id'],axis=1)
test = test.drop(['Id'],axis=1)

In [36]:
train.head()

,Altitude,Azimuthal_angle,Incline,H_dist_Hydro,V_dist_Hydro,H_dist_Road,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,H_dist_Fire,Wilderness_Area_A,Wilderness_Area_B,Wilderness_Area_C,Wilderness_Area_D,Soil,label
0,594,738,167,48,243,421,759,964,751,114,0,0,1,0,9,-1
1,343,103,212,43,239,25,861,818,480,208,0,0,1,0,1,-1
2,563,233,167,77,228,555,928,869,452,360,1,0,0,0,0,-1
3,715,602,136,193,289,175,830,979,684,164,0,0,1,0,17,-1
4,672,519,136,231,292,122,869,971,625,45,0,0,1,0,17,-1


In [37]:
test.head()

,Altitude,Azimuthal_angle,Incline,H_dist_Hydro,V_dist_Hydro,H_dist_Road,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,H_dist_Fire,Wilderness_Area_A,Wilderness_Area_B,Wilderness_Area_C,Wilderness_Area_D,Soil
0,489,355,197,237,333,456,948,909,460,404,1,0,0,0,3
1,626,333,257,192,337,398,971,881,393,165,0,0,1,0,16
2,657,103,242,137,252,236,857,802,464,636,0,1,0,0,8
3,543,175,182,232,365,500,905,846,456,317,1,0,0,0,0
4,746,888,121,150,261,197,783,916,680,274,0,0,1,0,16


In [38]:
val = train[10000:]
train = train[:10000]


In [39]:
y_train = train['label']
x_train = train.drop(['label'],axis=1)

y_val = val['label']
x_val = val.drop(['label'],axis=1)

In [40]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(10000, 15)
(10000,)
(2395, 15)
(2395,)


In [41]:
# Changing labels into 0 and 1 as the loss function 'binary_crossentropy' expects labels to be either 0 or 1

for i in range(len(y_train)): 
    if y_train[i]==-1:
        y_train[i]=0
    else:
        y_train[i]=1
print(y_train)



0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       1
10      0
11      0
12      0
13      1
14      0
15      0
16      1
17      0
18      0
19      0
20      0
21      1
22      0
23      0
24      0
25      0
26      1
27      0
28      0
29      0
       ..
9970    1
9971    0
9972    0
9973    0
9974    0
9975    0
9976    0
9977    0
9978    0
9979    0
9980    0
9981    0
9982    0
9983    0
9984    0
9985    0
9986    0
9987    0
9988    0
9989    1
9990    0
9991    0
9992    0
9993    1
9994    0
9995    0
9996    1
9997    0
9998    0
9999    0
Name: label, Length: 10000, dtype: int64


In [42]:
from sklearn.preprocessing import StandardScaler    # Using Standard Scalar to normalise our data
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.fit_transform(x_val)
test = sc.fit_transform(test)


/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input

In [43]:
model = Sequential()
model.add(Dense(32,activation='relu',input_shape=(15,))) #The number of rows (number of training samples) is not 
#the part of the input shape of the network because the training process feeds the network one
#sample per batch (or, more precisely, batch_size samples per batch).

model.add(Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [44]:
#Compiling the neural network
model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [45]:
model.fit(x_train, y_train, epochs=150, batch_size=30)


Epoch 1/150
10000/10000 [==============================] - 3s 262us/step - loss: 0.7710 - acc: 0.8430
Epoch 2/150
10000/10000 [==============================] - 2s 167us/step - loss: 0.4304 - acc: 0.8548
Epoch 3/150
10000/10000 [==============================] - 2s 176us/step - loss: 0.4078 - acc: 0.8544
Epoch 4/150
10000/10000 [==============================] - 2s 167us/step - loss: 0.4014 - acc: 0.8580
Epoch 5/150
10000/10000 [==============================] - 2s 173us/step - loss: 0.3993 - acc: 0.8580
Epoch 6/150
10000/10000 [==============================] - 2s 174us/step - loss: 0.3988 - acc: 0.8577
Epoch 7/150
10000/10000 [==============================] - 2s 179us/step - loss: 0.4007 - acc: 0.8561
Epoch 8/150
10000/10000 [==============================] - 2s 176us/step - loss: 0.3967 - acc: 0.8594
Epoch 9/150
10000/10000 [==============================] - 2s 171us/step - loss: 0.3963 - acc: 0.8598
Epoch 10/150
10000/10000 [==============================] - 2s 175us/step - loss: 

In [46]:
for i in range(10000,10000+len(y_val)):
    if y_val[i]==-1:
        y_val[i]=0
        

In [47]:
eval_model=model.evaluate(x_val, y_val)        # Validation accuracy
eval_model

2395/2395 [==============================] - 0s 112us/step


[0.3842061932748942, 0.8668058455861436]

In [48]:
pred = model.predict_classes(test)

In [49]:
for i in (pred):
    if i[0]==0:
        i[0]=-1
print(pred)
d = []
for i in pred:
    d.append(i[0])


[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


In [50]:

submission_df['Predicted'] = d
submission_df.to_csv('Keras_model.csv',index=False)

In [51]:
s = pd.read_csv('Keras_model.csv')
s.head(100)

,Id,Predicted
0,1,-1
1,2,-1
2,3,-1
3,4,-1
4,5,-1
5,6,-1
6,7,-1
7,8,-1
8,9,-1
9,10,-1
